In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
def expandText(inputText):
    model_name = 'doc2query/all-with_prefix-t5-base-v1'
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    prefix = "text2text"
    text = prefix+": "+inputText
    input_ids = tokenizer.encode(text, max_length=384, truncation=True, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=20)

    outputText = inputText

    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        outputText = outputText + ' ' +  query

    return outputText

Tia paola, mi cousing said a bad word, I can not say de word. I'm naked i can't say de word, mi cousing said a bad word, I can not say de word, I'm naked Yugioh, mi cousing said a bad word In a way, a girl can make a boy with bad words, because she's naked. I'm naked in front of my tv Tia paola, mi cousing said a bad word, I can not say de word, I can not say de word I'm naked without a reason. I can't say bad word Hi my cousin, I'm not a girl but I need advice Axed, I am in the lukewarm room I can't say a bad word TIAS, mi cousing said a bad word La mexicana - cuando que me puede ben en el acerco Tia paola, mi cousing said a bad word I'm naked but I'm naked I'm getting a bit afro at work today, Tia paola, mi cousing said a bad word, I can not say de word, I am naked Tia paola, mi cousing said a bad word, I can not say de word tia paola mi cousing said a bad word I'm naked, can you translate? Don't say a bad word in spanish.
